In [1]:
import numpy as np
import hydra
import torch
from omegaconf import OmegaConf
from pathlib import Path
from srcs.trainer import Trainer
from srcs.utils import instantiate, get_logger, is_master
import os
import shutil
from sklearn.model_selection import train_test_split

Разделение на трейн, тест и валидацию.

In [ ]:
data_dir = "./sign_language_dataset"

ttv_dataset_dit = "./ttv_dataset/"
train_dir = ttv_dataset_dit + "train"
test_dir = ttv_dataset_dit + "test"
val_dir = ttv_dataset_dit + "validation"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for class_folder in os.listdir(data_dir):
    print(class_folder)
    class_path = os.path.join(data_dir, class_folder)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        train_images, test_val_images = train_test_split(images, test_size=0.3, random_state=42)
        test_images, val_images = train_test_split(test_val_images, test_size=0.5, random_state=42)

        for image in train_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(train_dir, f"{class_folder}_{image}"))
        for image in test_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(test_dir, f"{class_folder}_{image}"))
        for image in val_images:
            shutil.copy(os.path.join(class_path, image), os.path.join(val_dir, f"{class_folder}_{image}"))

Обучение сети

In [ ]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)


def train_worker(config):
    if is_master():
        print(config)
    logger = get_logger('train')
    # setup data_loader instances
    data_loader, valid_data_loader = instantiate(config.data_loader)

    # build model. print it's structure and # trainable params.
    model = instantiate(config.arch)
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    logger.info(model)
    logger.info(f'Trainable parameters: {sum([p.numel() for p in trainable_params])}')

    # get function handles of loss and metrics
    criterion = instantiate(config.loss, is_func=True)
    metrics = [instantiate(met, is_func=True) for met in config['metrics']]

    # build optimizer, learning rate scheduler.
    optimizer = instantiate(config.optimizer, model.parameters())
    lr_scheduler = instantiate(config.lr_scheduler, optimizer)

    trainer = Trainer(model, criterion, metrics, optimizer,
                      config=config,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                      lr_scheduler=lr_scheduler)
    trainer.train()


def init_worker(working_dir, config):
    # initialize training config
    config = OmegaConf.create(config)
    config.cwd = working_dir
    # prevent access to non-existing keys
    OmegaConf.set_struct(config, True)
    # start training processes
    train_worker(config)


@hydra.main(version_base=None, config_path='conf/', config_name='train')
def main(config):
    n_gpu = torch.cuda.device_count()
    if config.gpu:
        config['n_gpu'] = n_gpu
    else:
        config['n_gpu'] = 0

    working_dir = str(Path.cwd().relative_to(hydra.utils.get_original_cwd()))
    if config.resume is not None:
        config.resume = hydra.utils.to_absolute_path(config.resume)
    config = OmegaConf.to_yaml(config, resolve=True)
    init_worker(working_dir, config)

if __name__ == '__main__':
    main()